In [2]:
import pandas as pd
from io import StringIO
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage, AIMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END, PauseNode, ProcessingNode
from langgraph.graph.message import add_messages
from pandas.core.frame import DataFrame
from pandas.core.series import Series
import json
# os.environ["OPENAI_API_KEY"] = getpass.getpass('Password: ')
api_key = "sk-pzmVDXe1-rxgtV34fEob57Owp9b7Z7_k-jYKmFaguDT3BlbkFJXR835WsPGSV7P_sjJXS2QGOJBIxgW5wSzwqExklvAA"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
class State(TypedDict):
    messages: list
    expected_info: dict

class Queryer:
    def __init__(self, llm, task_prompt: str):
        self.llm = llm
        self.task_prompt = task_prompt

    def __call__(self, state: State):
        lack_info_template = """'{key_name}' is not given"""
        lack_keys = []
        for k, v in state["expected_info"].items():
            if v is None:
                lack_keys.append(k)
        lack_info = "\n".join(lack_info_template.format(key_name=key_name) 
                              for key_name in lack_keys)
        system = "\n Currently absent information are:" + lack_info
        prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
        state["messages"].append((prompt | self.llm).invoke(self.task_prompt))
        return state
    
class Parser:
    def __init__(self, llm):
        self.llm = llm

    def __call__(self, state: State):
        message = state["messages"][-1] if len(state.get("messages", [])) > 0 else ""
        system = f"""You are a content parser. Your task is to extract values for specific keys from the provided content.
Return the result in this exact format:
key1,value1|key2,value2|...

Note: 1.Include only keys with found values in the response.
2. Do not add any extra information or explanations to the response.
3. Only one best value for each unique key.
        """
        key_string = ",".join(k for k in state["expected_info"].keys())
        system += "\nRequired keys are: \n" + key_string
        prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
        response = (prompt | self.llm).invoke(message.content)
        key_value_pairs = response.content.split('|')
        # TODO: modify the values for these keys and log the change into a message
        # we would like to print those changes in the screen when encountering it into iteration of the stream
        return state
    
#TODO: add pause node to wait for user input